# CoQuí - Correlated Quantum Interface

**CoQuí** is a C++ software project for *ab initio* electronic structure simulations beyond density functional theory (DFT) for quantum materials. 

---
Key features of CoQuí include: 
#### 🪚 Compressed Representations for Many-Body Hamiltonians
- Tensor Hypercontraction (THC) representation of two-electron Coulomb integrals [[ref1](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.3c00615), [ref2](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.4c00085)]
- Cholesky decomposition for two-electron integrals

#### ⚛️ Many-Body Perturbation Theory (MBPT)
- Hartree–Fock [[ref](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.3c00615)]
- RPA correlation energy [[ref](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.3c00615)]
- GW approximation [[ref](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.4c00085)]
- Second-order exchange (SOX) diagram [[ref](https://arxiv.org/abs/2404.17744)]
- Self-consistent MBPT:  
  - Quasiparticle approximation  
  - Full-frequency dependence
#### 🔻 Downfolding for Effective Low-Energy Hamiltonians
- Constrained RPA for calculating screened interactions [[ref](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.4c00085)]
- Construction of local effective Hamiltonians for correlated models [[ref](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.4c00085)]

---
CoQuí offers two modes of usage: 

1. **Input-file mode** $-$ Users parse a TOML input file and run CoQuí as a black-box executable. Output data are stored and exchanged via HDF5 files. 
2. **Python mode** $-$ Input parameters are specified directly in a Python program using dictionaries and passed to CoQui routines through thin Python wraoppers. 
   This mode enables more direct exchange of input and output data, allowing users to integrate CoQuí into their own computational workflows alongside external packages.

> ⚠️ **Note**: Even in Python mode, the user interacts with high-level interfaces rather than manipulating low-level components.
> CoQuí is **not** intended to serve as a low-level "toolbox" for building custom electronic structure algorithms from scratch.


## 0. Python setup
For setting up the environement for CoQuí Python interface, please add the following directory to your `PYTHONPATH`: 
```bash
export PYTHONPATH={COQUI_INSTALL_DIR}/lib/python/site-packages/:$PYTHONPATH
```

> ⚠️**Note**: The CoQuí C++ library requires an MPI environment and does not support serial execution (yet!). Therefore, the Python interface must be used within an initialized MPI context, such as one provided by mpi4py or another MPI-aware Python framework.


To run a Python script that uses CoQuí, use a command like: 
```bash
mpirun {MPI_FLAGS} python {YOUR_COQUI_PYTHON_SCRIPT}.py
```

In [1]:
import os
from mpi4py import MPI
import coqui

# CoQui version 
print(f"CoQui version: {coqui.__version__}")
# CoQui git information
print(f"Path for the CoQui Python library: {os.path.dirname(coqui.__file__)}")
print(f"GIT branch: {coqui.COQUI_GIT_BRANCH}")
print(f"GIT hash: {coqui.COQUI_GIT_HASH}\n")

--------------------------------------------------------------------------
Ignoring value for oob_tcp_if_exclude on worker6022 (10.250.112.0/20: Did not find interface matching this subnet).
(You can safely ignore this message.)
--------------------------------------------------------------------------


CoQui version: 0.1.0
Path for the CoQui Python library: /mnt/home/cyeh/Projects/BeyondDFT/build/genoa/pybind_llvm19/install/lib/python/site-packages/coqui
GIT branch: pybind
GIT hash: ab250de8



## 0.1 MPI envrionemnt

The state of the MPI environment in CoQuí is managed by the `coqui.MpiHandler` class. 
The class encapsulates information and provides access to 
- **Global** communicator $-$ All ranks 
- **Intranode** communicator $-$ Communicator within the same node
- **Internode** communicator $-$ Communicator between different nodes

In addition, `coqui.MpiHandler` offers a minimal interface for performing basic MPI operations. 

It must be constructed and, explicitly or implicitly, passed to **ALL** CoQuí routines that 
involve MPI parallelization. Even in serial mode (i.e., when using a single process), this class 
is required to ensure a consistent interface across all workflows.

> 💡**Warning**: The MpiHandler is not copiable. Attempting to copy it will fail:
> ```
>      import copy
>      mpi_copy = copy.copy(mpi)
> ```
>  Simply make another MpiHandler, rather than make a copy

In [3]:
# CoQuí computing environemnt: mpi handler and verbosity
coqui_mpi = coqui.MpiHandler()
coqui.set_verbosity(coqui_mpi, output_level=2, debug_level=0)
print(coqui_mpi)

CoQuí MPI state
-----------------
  Global communicator: rank 1 / 1
  Internode communicator: rank 1 / 1
  Intranode communicator: rank 1 / 1


## 0.2 Verbosity

The verbosity levels of all CoQuí routines are setup via `coqui.set_verbosity` function with the following parameters: 
   - mpi_handler:  [INPUT] MPI handler to ensure logging is MPI-aware, i.e. only root prints
   - output_level: [INPUT] Level of output verbosity (default: 2)
   - debug_level:  [INPUT] Level of debug verbosity (default: 0)


In [10]:
coqui.set_verbosity(coqui_mpi, output_level=2, debug_level=0)

## 1. CoQuí code structure 

To illustrate the code structure, here we show a standard usage of performing an *ab initio* Hartree-Fock calcualtion for silicon solid: 
```python
# Step 0: Set up the CoQuí computing environemnt: MPI handler and verbosity
mpi = coqui.MpiHandler()
coqui.set_verbosity(mpi, output_level=2, debug_level=0)

# Step 1: Construct a mean-field object
mf_params = {
    "prefix": "si", 
    "outdir": "data/qe_inputs/si/out", 
    "filetype": "h5"
}
si_mf = coqui.Mf(mpi, mf_params, "qe")

# Step 2: Compute THC-decomposed Coulomb integrals
eri_params = {
    "storage": "incore", "save": "thc.eri.h5",
    "ecut": 60, "thresh": 1e-2
}
si_thc = coqui.ThcCoulomb(si_mf, eri_params)

# Step 3: Run Hartree-Fock calculation
hf_params = {
    "restart": False, "output": "hf", "niter": 1,
    "beta": 300, "lambda": 1200, "iaft_prec": "mid",
    "iter_alg": {"alg": "damping", "mixing": 0.7}
}
coqui.run_hf(hf_params, h_int=si_thc)
```

Without diving into the details of each step, this calculation can be interpreted as three conceptual steps: 
1. **Initialize a Mean Field Object**:
   The `si_mf` object accesses data from a supported DFT code (Quantum ESPRESSO in this example),
   including system metadata and the single-particle basis set obtained from the mean-field solution. 
2. **Construct the Bare Coulomb Hamiltonian in THC Representation**:
   The `si_thc` object takes `si_mf` as input and constructs the bare Coulomb interaction using the
   tensor hypercontraction (THC) decomposition. 
4. **Perform the HF Self-Consistency Loop**:
   The function `run_hf` carries out the HF self-consistency using `si_thc` as the many-body Hamiltonian and
   `hf_params` as control parameters. 

---
#### 🔹 This example reflects a **general design pattern** used throughout CoQuí: 

>  **All electronic structure methods are implemented as free functions that accept**
>  - a **mean-field** object  
>  - a **Coulomb Hamiltonian** (e.g., in THC form)  
>  - a **dictionary of method-specific parameters**
>
>  These two core components—**mean-field** and **Coulomb Hamiltonian**—serve as the fundamental building blocks for composing many-body simulations.  

## 📚 2. Tutorial Overview

This tutorial series introduces the key features and capabilities of **CoQuí**, and is structured to reflect its modular design. 
You will begin with foundational components, such as DFT converter, mean-field construction, and Coulomb Hamiltonians, and then progress to many-body perturbation theories and downfolding techniques. 

#### 🧱 CoQuí Building Blocks

- **[Tutorial 0: DFT Converter](00b-DFT_Converter.ipynb)**  
  Convert DFT outputs into CoQuí-compatible formats.

- **[Tutorial 1: Mean-Field Class](01a-Mean_Field.ipynb)**  
  Learn how to construct and manipulate mean-field objects from DFT metadata.

- **[Tutorial 2: Bare Coulomb Hamiltonian](02a-THC_Coulomb_Hamiltonian.ipynb)**  
  Compute and store the two-electron Coulomb integrals in compressed form.

#### 🔬 Many-Body Electronic Structure

- **[Tutorial 3: Many-Body Perturbation Theory (MBPT)](02-MBPT.ipynb)**  
  Perform HF, RPA, GW, and SOX calculations with different self-consistency.

- **[Tutorial 4: Downfolding for Screened Interactions](03b-Downfold_Coulomb.ipynb)**  
  Generate screened interactions using constrained RPA and construct low-energy models.

- **[Tutorial 5: Interfacing with ModEST](03c-ModEST.ipynb)**  
  Export CoQuí-generated Hamiltonians for external solvers such as ModEST.

- **[Tutorial 6: GW+EDMFT](04-GW_EDMFT.ipynb)**  
  Combine long-range GW screening with local strong correlations in a self-consistent framework.

---

> ⚠️ These tutorials are designed to be **sequantial**.
> The order is intentional, as each topic builds upon the previous one. Please follow them in the designated order for a coherent understanding. 